In [63]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from lib import jsoncorpus, datastuff
import gensim
import traceback
import numpy as np
import matplotlib.pyplot as plt
import sklearn.svm
import sklearn.feature_extraction

# LDA

In [ ]:
# Load in the previously created corpus and dictionary of scraped sites
dictionary, corpus, meta_corpus = jsoncorpus.load_or_create('docs/sites.jl')

Building new dictionary
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
Building new corpora


In [ ]:
# Build a list of all topics
alltopics = []
toptopics = []
# And a list linking website URLs to DMOZ categories
meta = []

for site in datastuff.loadSplitJsonLines('docs/sites.jl.'):
    alltopics.append(', '.join(site['topics']))
    toptopics.append(site['topics'][0])
    meta.append((site['url'], site['topics']))
alltopics = set(alltopics)
toptopics = set(toptopics)

In [16]:
# Generate LDA Model of document bodies
model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics = len(alltopics), passes = 1, chunksize = 100)

In [41]:
# Generate LDA Model of metadata
meta_model = gensim.models.ldamodel.LdaModel(corpus=meta_corpus, id2word=dictionary, num_topics = len(toptopics), passes = 1, chunksize = 100)

In [43]:
# Show some examples of LDA topics
for t in meta_model.show_topics(10)[:5]:
    print t

0.034*products + 0.031*party + 0.016*trade + 0.015*architecture + 0.015*english + 0.015*company + 0.014*libraries + 0.013*center + 0.012*engineering + 0.012*university
0.038*resources + 0.029*services + 0.026*web + 0.021*maps + 0.020*map + 0.018*travel + 0.018*software + 0.015*city + 0.015*data + 0.013*air
0.074*kids + 0.069*books + 0.044*american + 0.031*school + 0.015*articles + 0.014*family + 0.012*government + 0.012*great + 0.012*disney + 0.009*puppets
0.082*art + 0.016*parents + 0.016*forum + 0.014*card + 0.013*directory + 0.012*words + 0.011*quotes + 0.011*source + 0.010*global + 0.009*beauty
0.066*health + 0.054*news + 0.033*medical + 0.024*time + 0.018*world + 0.017*thesaurus + 0.015*national + 0.013*business + 0.012*adventure + 0.011*latin


In [55]:
# Show some examples of topics for a particular corpus
site = 650
print meta[site]
print 
topics = [(p, [x[1] for x in model.show_topic(tid)[:6]]) for tid, p in model[corpus[site]]]
topics = sorted(topics, key = (lambda x: -x[0]))
sums = 0
for x in topics:
    sums += x[0]
    if sums > 0.9:
        break
    print x
print "\nTotal prob: {}\n\n".format(sums)


(u'http://www.fantasyleague.com/', [u'sports', u'fantasy'])

(0.11147272064363745, [u'february', u'january', u'march', u'prize', u'year', u'november'])
(0.10440393001192687, [u'term', u'world', u'ancient', u'original', u'early', u'set'])
(0.094531712338769266, [u'win', u'beat', u'county', u'match', u'wins', u'left'])
(0.076165698742543878, [u'studies', u'graphic', u'virtual', u'study', u'eye', u'partners'])
(0.069068103836328734, [u'conference', u'league', u'1990s', u'social', u'preview', u'intended'])
(0.051561780975683799, [u'series', u'week', u'happy', u'night', u'opened', u'unique'])
(0.051543621165325237, [u'draw', u'exercise', u'cup', u'premier', u'world', u'finder'])
(0.044215058173857319, [u'top', u'sports', u'stories', u'sign', u'visual', u'entertainment'])
(0.032136221767790497, [u'press', u'release', u'complete', u'highlights', u'introductory', u'february'])
(0.029783411662020427, [u'awards', u'players', u'registration', u'event', u'annual', u'teams'])
(0.025723733353203937,

# Category assignment

In [52]:
model[corpus[1:3]]

In [ ]:
se

In [16]:
# Build an array of features linking corpus to topics
features = np.zeros((len(corpus), len(alltopics)))
totals = np.zeros(len(alltopics))
for i, doc in enumerate(model[corpus]):
    for topic, p in doc:
        features[i, topic] = p
        totals[topic] += p
        


In [ ]:
plt.imshow(features.T)

In [70]:
import sklearn.preprocessing
import sklearn.tree
import sklearn.svm
import sklearn.multiclass
import sklearn.ensemble
import sklearn.cross_validation


all_ylabels = np.array([x['topics'][:2] for x in datastuff.loadSplitJsonLines('docs/sites.jl.')])
import itertools
import collections
counts = collections.Counter(itertools.chain.from_iterable(all_ylabels))
uncommon = [x for x in counts if counts[x] < 9]
ylabels = np.array([[i for i in l if i not in uncommon] for l in all_ylabels])

#if len(ylabels.shape) == 1:
if False:
    label_processor = sklearn.preprocessing.LabelEncoder()
else:
    label_processor = sklearn.preprocessing.MultiLabelBinarizer()

label_processor.fit(ylabels)
#clf = sklearn.tree.DecisionTreeClassifier(random_state=0)
clf = sklearn.ensemble.ExtraTreesClassifier(random_state=0, n_estimators=100, oob_score=True, bootstrap=True, n_jobs=4)
X = features[:,np.sum(features > 0, axis=0)>0]
#X = np.random.rand(X.shape[0], X.shape[1])
y = label_processor.transform(ylabels)
print sklearn.cross_validation.cross_val_score(clf, X, y)


[ 0.00375     0.00875     0.00375469]


In [71]:
label_processor.transform(ylabels[1:4])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  

In [72]:
ylabels[1:40]

array([[u'sports', u'software'], [u'sports', u'software'],
       [u'sports', u'software'], [u'sports', u'software'],
       [u'sports', u'software'], [u'sports', u'people'],
       [u'sports', u'organizations'], [u'sports', u'organizations'],
       [u'sports', u'organizations'], [u'sports', u'software'],
       [u'sports', u'software'], [u'sports'],
       [u'sports', u'organizations'], [u'sports', u'organizations'],
       [u'sports'], [u'sports', u'organizations'],
       [u'sports', u'organizations'], [u'sports', u'organizations'],
       [u'sports', u'organizations'], [u'sports'], [u'sports'],
       [u'sports'], [u'sports', u'winter sports'],
       [u'sports', u'winter sports'], [u'sports', u'water sports'],
       [u'sports', u'water sports'], [u'sports', u'water sports'],
       [u'sports', u'water sports'], [u'sports', u'organizations'],
       [u'sports', u'walking'], [u'sports', u'walking'],
       [u'sports', u'walking'], [u'sports', u'walking'],
       [u'sports', u'walk

In [73]:
ylabels[100]

[u'sports', u'resources']

In [54]:
test = sklearn.preprocessing.MultiLabelBinarizer().fit(ylabels)
print test.transform(ylabels[1])
print test.inverse_transform(test.transform([ylabels[0]]))


NameError: name 'ylabels' is not defined

In [75]:
#zip(clf.transform(X)[0][np.where(clf.transform(X)[0]>0)], ylabels[np.where(clf.transform(X)[0]>0)])
clf.fit(X, y)

ExtraTreesClassifier(bootstrap=True, compute_importances=None,
           criterion='gini', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
           min_samples_split=2, n_estimators=100, n_jobs=4, oob_score=True,
           random_state=0, verbose=0)

In [77]:
meta[0:10]

[(u'http://www.siliconcoach.com/', [u'sports', u'software']),
 (u'http://www.recenroll.com/', [u'sports', u'software']),
 (u'http://www.gametime.net/', [u'sports', u'software']),
 (u'http://www.foxsportspulse.com/', [u'sports', u'software']),
 (u'http://www.crosstrainer.ca/', [u'sports', u'software']),
 (u'http://www.integratedsports.net/', [u'sports', u'software']),
 (u'http://www.sporting-heroes.net/', [u'sports', u'people']),
 (u'http://coedjewishsports.org/', [u'sports', u'organizations']),
 (u'http://www.sportandrecreation.org.uk', [u'sports', u'organizations']),
 (u'http://www.eis2win.co.uk/pages/default.aspx',
  [u'sports', u'organizations'])]

In [76]:
from pprint import pprint
pprint(label_processor.inverse_transform(clf.predict(X[0:10])))


[(u'software', u'sports'),
 (u'software', u'sports'),
 (u'software', u'sports'),
 (u'software', u'sports'),
 (u'software', u'sports'),
 (u'software', u'sports'),
 (u'people', u'sports'),
 (u'organizations', u'sports'),
 (u'organizations', u'sports'),
 (u'organizations', u'sports')]


In [16]:
X_train, X_test, y_train, y_test, ylabels_train, ylabels_test, meta_train, meta_test, ind_train, ind_test = sklearn.cross_validation.train_test_split(X, y, ylabels, meta, range(len(y)))
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

derp1 = np.array([t[0] for t in meta_test])[:15]
derp2 = dv.inverse_transform(pred)[:15]
derp3 = np.array([t[0] for t in meta_test])[:15]

#print np.vstack(()).T
#print ylabels[testnum]


NameError: name 'dv' is not defined

In [ ]:
print meta[897]
print [dictionary[x[0]] for x in corpus[897]]

In [ ]:
print dv.inverse_transform(pred)
print ylabels[testnum]

In [ ]:
' '.join(dictionary[w] for w, f in corpus[site])

In [ ]:
hdpmodel = gensim.models.hdpmodel.HdpModel(corpus=corpus, id2word=dictionary)

In [ ]:
hdpmodel[corpus[5]]

In [ ]:
hdptops = hdpmodel.show_topics()
[x[0] for x in hdpmodel[corpus[5]]]

In [ ]:
hdptops[8]


In [ ]:
hdpmodel.m_eta + hdpmodel.m_lambda
hdpmodel.id2word